In [2]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
from pandas import read_parquet

In [5]:
df_bike=pd.read_parquet('./data_backup/bike_final_data')
df_taxi=pd.read_parquet('./data_backup/taxi_final_data')
df_join=pd.read_csv('./data_backup/bike-zone.csv')
df_volume=pd.read_parquet('volume_final_data')
df_join2=pd.read_csv('./data_backup/volume_zone (2).csv')

In [3]:
df_bike.head(2)

,datetime_rounded,station id,busyness
0,2018-01-25 08:30:00,3472.0,0.182482
1,2018-01-17 18:30:00,3472.0,0.182482


In [6]:
df_bike=df_bike.rename(columns={'datetime_rounded':'datetime','busyness':'bike_busyness'})
df_bike.head(2)

,datetime,station id,bike_busyness
0,2018-01-25 08:30:00,3472.0,0.182482
1,2018-01-17 18:30:00,3472.0,0.182482


In [5]:
print(df_bike['bike_busyness'].max())
print(df_bike['bike_busyness'].min())
print(df_bike['bike_busyness'].median())

1.0
0.0
0.10948905109489052


In [7]:
df_taxi=df_taxi.rename(columns={'LocationID':'Taxi_Zone_ID','busyness':'taxi_busyness'})
df_taxi.head(2)

,datetime,Taxi_Zone_ID,taxi_busyness
0,2018-01-01 00:36:02,4,0.340029
1,2018-01-01 01:14:30,4,0.340029


In [8]:
df_taxi['datetime'] = df_taxi['datetime'].dt.floor('15min')
df_taxi.head(5)

,datetime,Taxi_Zone_ID,taxi_busyness
0,2018-01-01 00:30:00,4,0.340029
1,2018-01-01 01:00:00,4,0.340029
2,2018-01-01 01:00:00,4,0.340029
3,2018-01-01 00:30:00,4,0.340029
4,2018-01-01 00:30:00,4,0.340029


In [34]:
print(df_taxi['taxi_busyness'].min())
print(df_taxi['taxi_busyness'].max())
print(df_taxi['taxi_busyness'].median())

0.0
1.0
0.09612625538020085


In [35]:
df_volume.head(2)

,SegmentID,datetime,busyness
0,70896,2018-03-03 00:00:00,0.003418
1,70896,2018-03-03 00:15:00,0.003418


In [9]:
df_volume=df_volume.rename(columns={'busyness':'volume_busyness'})
df_volume.head(2)

,SegmentID,datetime,volume_busyness
0,70896,2018-03-03 00:00:00,0.155462
1,70896,2018-03-03 00:15:00,0.155462


In [10]:
print(df_volume['volume_busyness'].min())
print(df_volume['volume_busyness'].max())
print(df_volume['volume_busyness'].median())

0.0
1.0
0.006623521084032764


In [38]:
df_join.head(2)

,Taxi_Zone_ID,station id
0,4,302.0
1,4,150.0


In [39]:
df_join2.head(2)

,Taxi_Zone_ID,SegmentID
0,4,34607
1,4,33218


In [45]:
df_taxi.shape

(7634627, 3)

## check duplicate before merging

bike data

In [11]:
df_bike.shape

(17545870, 3)

In [11]:
bike_dupliate=df_bike.duplicated()

In [12]:
bike_dupliate.sum()

7399458

In [12]:
df_bike=df_bike.drop_duplicates()
df_bike.shape

(10146412, 3)

In [13]:
# 创建日期序列，覆盖所需的时间范围
start_date = df_bike['datetime'].min().date()
end_date = df_bike['datetime'].max().date()
date_range = pd.date_range(start=start_date, end=end_date, freq='D')

# 查找缺失数据
missing_dates = date_range[~date_range.isin(df_bike['datetime'])]

# 打印缺失数据
print(missing_dates)

DatetimeIndex(['2018-03-22', '2018-12-02', '2018-12-03', '2018-12-04'], dtype='datetime64[ns]', freq=None)


taxi data

In [15]:
df_taxi.shape

(7634627, 3)

In [98]:
df_taxi.duplicated().sum()

5099726

In [17]:
df_taxi_clean=df_taxi.drop_duplicates()

In [18]:
start_date = pd.Timestamp('2018-01-01').date()
end_date = pd.Timestamp('2018-12-31').date()
date_range_2018 = pd.date_range(start=start_date, end=end_date, freq='D')
all_days_present = all(date in df_taxi_clean['datetime'].unique() for date in date_range_2018)
print(all_days_present)


True


volume data

In [14]:
df_volume.shape

(497668, 3)

In [106]:
df_volume.duplicated().sum()

5805

# merge

In [26]:
df_bike.shape

(10146412, 3)

In [15]:
merge1=pd.merge(df_bike,df_join,on='station id')

In [25]:
merge1.shape

(7417046, 4)

In [29]:
start_date = pd.Timestamp('2018-01-01').date()
end_date = pd.Timestamp('2018-12-31').date()
date_range_2018 = pd.date_range(start=start_date, end=end_date, freq='D')
all_days_present = all(date in merge1['datetime'].unique() for date in date_range_2018)
print(all_days_present)


False


In [31]:
merge1.head(10)

,datetime,station id,bike_busyness,Taxi_Zone_ID
0,2018-01-25 08:30:00,3472.0,0.182482,68
1,2018-01-17 18:30:00,3472.0,0.182482,68
2,2018-01-19 18:15:00,3472.0,0.182482,68
3,2018-01-13 03:00:00,3472.0,0.182482,68
4,2018-01-03 17:30:00,3472.0,0.182482,68
5,2018-01-10 14:15:00,3472.0,0.182482,68
6,2018-01-31 17:45:00,3472.0,0.182482,68
7,2018-01-24 15:15:00,3472.0,0.182482,68
8,2018-01-27 13:00:00,3472.0,0.182482,68
9,2018-01-16 12:45:00,3472.0,0.182482,68


In [16]:
merge2=pd.merge(merge1,df_taxi,on=('datetime','Taxi_Zone_ID'))

In [38]:
merge2.shape

(25547447, 5)

In [64]:
# threshold = 0.3  # 重复数据的阈值
# duplicate_rows = merge1[merge1.duplicated()]  # 找到所有重复的行
# num_duplicates = len(duplicate_rows)  # 重复行的数量
# num_rows = len(merge1)  # DataFrame 的总行数

# if num_duplicates > 0:
#     # 计算需要删除的重复数据的数量
#     num_to_drop = int(num_duplicates * threshold)
#     # 随机选择要删除的重复行
#     rows_to_drop = duplicate_rows.sample(n=num_to_drop, random_state=42)
#     # 删除重复行
#     merge1_cleaned = merge1.drop(rows_to_drop.index)
# else:
#     merge1_cleaned = merge1.copy()

# # 打印处理后的 DataFrame
# merge1_cleaned.shape

(17545870, 3)

In [59]:
# merge2.duplicated().sum()

6722023

In [62]:
# merge2=merge2.drop_duplicates()

In [39]:
merge2.shape

(25547447, 5)

In [17]:
merge3=pd.merge(df_volume,df_join2,on='SegmentID')

In [41]:
merge3.shape

(442704, 4)

In [68]:
merge3.duplicated().sum()

5678

In [18]:
merge3=merge3.drop_duplicates()

In [49]:
merge3.dtypes

SegmentID                   int64
datetime           datetime64[ns]
volume_busyness           float64
Taxi_Zone_ID                int16
dtype: object

In [50]:
merge2.dtypes

datetime         datetime64[ns]
station id              float64
bike_busyness           float64
Taxi_Zone_ID              int16
taxi_busyness           float64
dtype: object

In [19]:
merge3['Taxi_Zone_ID']=merge3['Taxi_Zone_ID'].astype('int16')
# merge3['datetime']=merge3['datetime'].astype('datetime32[D]')
merge3['SegmentID']=merge3['SegmentID'].astype('int16')
# merge3['volume_busyness']=merge3['volume_busyness'].astype('float32')

merge2['Taxi_Zone_ID']=merge2['Taxi_Zone_ID'].astype('int16')
# merge2['datetime']=merge2['datetime'].astype('datetime32[D]')
merge2['station id']=merge2['station id'].astype('int16')
# merge2['bike_busyness']=merge2['bike_busyness'].astype('float64')
# merge2['taxi_busyness']=merge2['taxi_busyness'].asfreq('float32')

In [20]:
final_merge=pd.merge(merge2,merge3,on=('datetime','Taxi_Zone_ID'))

In [21]:
final_merge.shape

(8625761, 7)

In [22]:
final_merge.to_parquet('data_for_model')

In [64]:
start_date = final_merge['datetime'].median().date()
print(start_date)

2018-10-21


In [60]:
import pandas as pd

# 创建日期序列，覆盖所需的时间范围
start_date = final_merge['datetime'].min().date()
end_date = final_merge['datetime'].max().date()
date_range = pd.date_range(start=start_date, end=end_date, freq='D')

# 查找缺失数据
missing_dates = date_range[~date_range.isin(final_merge['datetime'])]

# 打印缺失数据
print(missing_dates)


DatetimeIndex(['2018-01-16', '2018-01-18', '2018-02-02', '2018-02-04',
               '2018-02-05', '2018-02-07', '2018-02-08', '2018-02-09',
               '2018-02-10', '2018-02-11',
               ...
               '2018-10-03', '2018-10-04', '2018-10-06', '2018-10-07',
               '2018-10-08', '2018-10-09', '2018-10-10', '2018-10-11',
               '2018-10-12', '2018-10-13'],
              dtype='datetime64[ns]', length=179, freq=None)
